# Gradient Boosting Regression

Demonstrate Gradient Boosting on the Boston housing dataset.

This example fits a Gradient Boosting model with least squares loss and 500 regression trees of depth 4.

This is a port to OCaml of the [scikit-learn gradient boosting regression example](https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py).

In [ ]:
#require "pyml"
#require "matplotlib"
#require "jupyter.notebook"
#require "shell"

In [ ]:
open Matplotlib;;
let plot () =
  let data = Mpl.plot_data `png in
  ignore (Jupyter_notebook.display ~base64:true "image/png" data);;

let () =
    Mpl.set_backend Agg
;;

In [ ]:
(* load sklearn from the current git repo (build first with dune build @install) *)
let root = String.trim @@ Shell.run_full "git" ["rev-parse"; "--show-toplevel"];;
let libdir = root ^ "/_build/install/default/lib/sklearn/";;
Topdirs.dir_directory libdir;;
#load "sklearn.cma";;

In [ ]:
(* it would be nice if the OCaml kernel installed printers automatically like utop *)
#install_printer Sklearn.Arr.pp;;
#install_printer Sklearn.Ndarray.pp;;
#install_printer Sklearn.Ensemble.GradientBoostingRegressor.pp;;

## Load data

In [ ]:
(* The Python source does a custom split using shuffle(). The equivalent with train_test_split is simpler (see below) ,
   but it is nice to reproduce the numerical results and graphics. *)
   
(* (* the recommended simpler version *)
  let [@ocaml.warning "-8"] [x_train; x_test; y_train; y_test] =
  Sklearn.Model_selection.train_test_split [boston#data; boston#target] ~random_state:42 ~train_size:(`F 0.9);; *)

let boston = Sklearn.Datasets.load_boston();;

let [@ocaml.warning "-8"] [x; y] = Sklearn.Utils.shuffle [boston#data; boston#target] ~random_state:13;;
let offset = int_of_float @@ (float_of_int (Sklearn.Arr.shape x).(0)) *. 0.9;;
let x_train = Sklearn.Arr.(get_sub [slice ~j:offset ()] x);;
let y_train = Sklearn.Arr.(get_sub [slice ~j:offset ()] y);;
let x_test = Sklearn.Arr.(get_sub [slice ~i:offset ()] x);;
let y_test = Sklearn.Arr.(get_sub [slice ~i:offset ()] y);;

## Fit regression model

In [ ]:
let get_f = function `F x -> x | _ -> assert false;;

let n_estimators = 500;;

module Gbr = Sklearn.Ensemble.GradientBoostingRegressor;;
let clf =
  Gbr.(create ~n_estimators ~max_depth:4 ~min_samples_split:(`I 2) ~learning_rate:0.01 ~loss:`Ls()
       |> fit ~x:x_train ~y:y_train);;

let mse = Sklearn.Metrics.mean_squared_error ~y_true:y_test ~y_pred:(Gbr.predict clf ~x:x_test) () |> get_f in
Printf.printf "MSE: %.4f\n%!" mse;;

## Plot training deviance

In [ ]:
let protect f =
  try f()
  with (Py.E (a, b)) as exc -> Printf.printf "error: %s\n%s\n%!" (Py.Object.to_string a) (Py.Object.to_string b); raise exc

let call_loss ~y ~raw_predictions loss =
  Py.Callable.to_function loss [|Sklearn.Arr.to_pyobject y; Sklearn.Arr.to_pyobject raw_predictions|] |> Py.Float.to_float;;

(* TODO: contribute this to ocaml-matplotlib *)
let set_yticks ax ticks =
  let _ = Py.Module.get_function (Ax.Expert.to_pyobject ax) "set_yticks" [|Sklearn.Arr.to_pyobject ticks|] in ();;

let set_yticklabels ax labels =
  let _ = Py.Module.get_function (Ax.Expert.to_pyobject ax) "set_yticklabels" [|Sklearn.Arr.to_pyobject labels|] in ();;

(* Axes.barh(self, y, width, height=0.8, left=None, *, align='center', **kwargs)[source]) *)
let barh ax y width =
  let _ = Py.Module.get_function (Ax.Expert.to_pyobject ax) "barh" [|Sklearn.Arr.to_pyobject y; Sklearn.Arr.to_pyobject width|] in ();;

(* plot deviance *)
(* TODO: better wrapper for staged_predict (return iterator of Arr.t) *)
(* TODO: better wrapper for Gbr.loss_: return callable function *)
let test_score =
  let score = Sklearn.Ndarray.zeros ~dtype:(`S "float64") [n_estimators] in
  let _ = Py.Iter.fold_left
    (fun i e -> Sklearn.Ndarray.set [|`I i|] (`F (call_loss y_test (Sklearn.Arr.of_pyobject e) (Gbr.loss_ clf))) score; i+1)
    0 (Gbr.staged_predict clf ~x:x_test)
  in score
in

let fig, ax1, ax2 = Fig.create_with_two_axes ~figsize:(12., 6.) `horizontal in

let xs = Array.init n_estimators (fun i -> float_of_int (i+1)) in
let train_score = Gbr.train_score_ clf |> Sklearn.Arr.get_ndarray |> Sklearn.Ndarray.to_float_array in
Ax.set_title ax1 "Deviance";
Ax.plot ax1 ~label:"Training Set Deviance" ~linestyle:Solid ~xs train_score;
Ax.plot ax1 ~label:"Test Set Deviance" ~linestyle:Solid ~xs (Sklearn.Ndarray.to_float_array test_score);
Ax.legend ax1;
Ax.set_xlabel ax1 "Boosting iterations";
Ax.set_ylabel ax1 "Deviance";

(* TODO: rethink Ops (this should include min/max/argsort...?) *)
let feature_importance = Gbr.feature_importances_ clf in
let feature_importance = Sklearn.Arr.Ops.((int 100) * feature_importance / (float (Sklearn.Arr.max feature_importance))) in
let sorted_idx = Sklearn.Arr.argsort feature_importance in
let pos = Sklearn.Arr.arange (Sklearn.Arr.shape sorted_idx).(0) in
let pos = Sklearn.Arr.Ops.(pos + (float 0.5)) in
Ax.set_title ax2 "Variable Importance";
barh ax2 pos Sklearn.Arr.(get_sub [`Arr sorted_idx] feature_importance);
set_yticks ax2 pos;
set_yticklabels ax2 (Sklearn.Arr.get_sub [`Arr sorted_idx] boston#feature_names);
Ax.set_xlabel ax2 "Relative Importance";
Ax.set_title ax2 "Variable Importance";
plot ();;